# Calculations

---

## Short Circuit [Code](https://github.com/FlorianShepherd/pandapower-youtube/blob/master/scripts/short_circuit.py)

<a href="http://www.youtube.com/watch?feature=player_embedded&v=66SSDhSNV7k" target="_blank">
    <img 
        src="http://img.youtube.com/vi/66SSDhSNV7k/0.jpg" 
        alt="Easy plotting Youtube Video" width="240" height="180" border="10" 
    />
</a>

> Watch video to understand short circuit in detail

In [9]:
import pandapower as pp
import pandapower.shortcircuit as sc

def ring_network():
    net = pp.create_empty_network()
    b1 = pp.create_bus(net, 220)
    b2 = pp.create_bus(net, 110)
    b3 = pp.create_bus(net, 110)
    b4 = pp.create_bus(net, 110)

    pp.create_ext_grid(net, b1, s_sc_max_mva=100., s_sc_min_mva=80., rx_min=0.20, rx_max=0.35)

    pp.create_transformer(net, b1, b2, "100 MVA 220/110 kV")
    pp.create_line(net, b2, b3, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV", length_km=15.)
    l2 = pp.create_line(net, b3, b4, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV", length_km=12.)
    pp.create_line(net, b4, b2, std_type="N2XS(FL)2Y 1x120 RM/35 64/110 kV", length_km=10.)
    pp.create_switch(net, b4, l2, closed=False, et="l")
    return net

net = ring_network()
sc.calc_sc(net)
print(net.res_bus_sc)

    ikss_ka     skss_mw      rk_ohm      xk_ohm
0  0.262432  100.000000  175.878566  502.510189
1  0.476454   90.776637   44.276323  139.778739
2  0.466671   88.912758   46.571323  142.268739
3  0.469892   89.526397   45.806323  141.438739


In [6]:
# To calculate "thermal current"
# For thermal current, we have to set time tk_s, which is by default 2 and if we increase it, 
# thermal current will be higher
sc.calc_sc(net, ith=True, tk_s=2.)
print(net.res_bus_sc)

    ikss_ka     skss_mw    ith_ka      rk_ohm      xk_ohm
0  0.262432  100.000000  0.263078  175.878566  502.510189
1  0.476454   90.776637  0.477748   44.276323  139.778739
2  0.466671   88.912758  0.467898   46.571323  142.268739
3  0.469892   89.526397  0.471141   45.806323  141.438739


In [7]:
# To calculate "peak current"
sc.calc_sc(net, ip=True)
print(net.res_bus_sc)

    ikss_ka     skss_mw     ip_ka      rk_ohm      xk_ohm
0  0.262432  100.000000  0.505834  175.878566  502.510189
1  0.476454   90.776637  0.942589   44.276323  139.778739
2  0.466671   88.912758  0.915418   46.571323  142.268739
3  0.469892   89.526397  0.924301   45.806323  141.438739


In [9]:
# To get results for lines
sc.calc_sc(net, branch_results=True)
print(net.res_line_sc)

Branch results are in beta mode and might not always be reliable, especially for transformers


    ikss_ka
0  0.466671
1  0.000000
2  0.469892


The above one is `Ring Network` but if we close the switches, we get `Meshed Network` and the calculation changes a little bit and now you will have automatic topology detection

In [13]:
# If we close all switches in the network, topology changes to mesh grid and automatically detected
net = ring_network()
net.switch.loc[:, "closed"] = True
sc.calc_sc(net, ith=True, ip=True, tk_s=2., branch_results=True, topology="auto")
print(net.res_bus_sc)
print(net.res_line_sc)

Branch results are in beta mode and might not always be reliable, especially for transformers


    ikss_ka     skss_mw     ip_ka    ith_ka      rk_ohm      xk_ohm
0  0.262432  100.000000  0.505834  0.263078  175.878566  502.510189
1  0.476454   90.776637  0.942589  0.477748   44.276323  139.778739
2  0.470593   89.660073  0.926244  0.471847   45.640917  141.259279
3  0.471649   89.861256  0.929174  0.472910   45.392809  140.990090
    ikss_ka     ip_ka    ith_ka
0  0.279812  0.550740  0.280558
1  0.190781  0.375504  0.191289
2  0.344176  0.678046  0.345097


---

## State Estimation [Code](https://github.com/FlorianShepherd/pandapower-youtube/blob/master/scripts/simple_state_estimation.py)

<a href="http://www.youtube.com/watch?feature=player_embedded&v=p8re_RzmlbU" target="_blank">
    <img 
        src="http://img.youtube.com/vi/p8re_RzmlbU/0.jpg" 
        alt="Easy plotting Youtube Video" width="240" height="180" border="10" 
    />
</a>

> Want to know voltage at each bus in the system, __definitely watch video__

In [14]:
import pandapower.estimation as est
import pandapower as pp

net = pp.create_empty_network()

b1 = pp.create_bus(net, name="bus 1", vn_kv=1., index=1)
b2 = pp.create_bus(net, name="bus 2", vn_kv=1., index=2)
b3 = pp.create_bus(net, name="bus 3", vn_kv=1., index=3)

pp.create_ext_grid(net, b1)  # set the slack bus to bus 1

l1 = pp.create_line_from_parameters(net, 1, 2, 1, r_ohm_per_km=.01, x_ohm_per_km=.03, 
                                    c_nf_per_km=0, max_i_ka=1)
l2 = pp.create_line_from_parameters(net, 1, 3, 1, r_ohm_per_km=.02, x_ohm_per_km=.05, 
                                    c_nf_per_km=0, max_i_ka=1)
l3 = pp.create_line_from_parameters(net, 2, 3, 1, r_ohm_per_km=.03, x_ohm_per_km=.08, 
                                    c_nf_per_km=0, max_i_ka=1)

# Measure bus voltages
# params network, measurement of type voltage, for a bus, with this measurement value from device, 
# error of measuring device, for bus 1
pp.create_measurement(net, "v", "bus", 1.006, 0.004, b1)
pp.create_measurement(net, "v", "bus", .968, 0.004, b2)

# measure bus p(MW), q(MVar)
# error in kV, kVar
pp.create_measurement(net, "p", "bus", -0.501, 10., b2)
pp.create_measurement(net, "q", "bus", -.266, 10., b2)

# line measurements
pp.create_measurement(net, "p", "line", 0.888, 8., l1, side=1)
pp.create_measurement(net, "p", "line", 1.173, 8., l2, side=1)
pp.create_measurement(net, "q", "line", 0.568, 8., l1, side=1)
pp.create_measurement(net, "q", "line", 0.663, 8., l2, side=1)

7

In [15]:
# We have 4 bus measurements and 4 line measurements
# Now, we can perform state estimation
est.estimate(net)

True

In [16]:
# Estimated value for each bus in system
print(net.res_bus_est)

      vm_pu  va_degree      p_mw    q_mvar
1  1.005989   0.000000 -2.184483 -2.058855
2  0.968011  -0.724121  0.158678  0.694332
3  0.927532  -3.130342  1.934930  1.127138


In [17]:
# Estimated line measurements
print(net.res_line_est)

   p_from_mw  q_from_mvar   p_to_mw  q_to_mvar     pl_mw   ql_mvar  i_from_ka  \
0   0.752046     1.025443 -0.736067  -0.977505  0.015979  0.047937   0.729819   
1   1.432437     1.033412 -1.370782  -0.879273  0.061656  0.154139   1.013703   
2   0.577388     0.283173 -0.564148  -0.247865  0.013240  0.035308   0.383558   

    i_to_ka      i_ka  vm_from_pu  va_from_degree  vm_to_pu  va_to_degree  \
0  0.729819  0.729819    1.005989        0.000000  0.968011     -0.724121   
1  1.013703  1.013703    1.005989        0.000000  0.927532     -3.130342   
2  0.383558  0.383558    0.968011       -0.724121  0.927532     -3.130342   

   loading_percent  
0        72.981922  
1       101.370306  
2        38.355775  


---

## Optimal Power Flow [Code](https://github.com/FlorianShepherd/pandapower-youtube/blob/master/scripts/simple_opf.py)

<a href="http://www.youtube.com/watch?feature=player_embedded&v=-NZHOlRYuzM" target="_blank">
    <img 
        src="http://img.youtube.com/vi/-NZHOlRYuzM/0.jpg" 
        alt="Easy plotting Youtube Video" width="240" height="180" border="10" 
    />
</a>

In [26]:
import pandapower as pp

net = pp.create_empty_network()

min_vm_pu = .95  # Minimum bus voltage
max_vm_pu = 1.05  # Maximum bus voltage

# create buses
bus1 = pp.create_bus(net, vn_kv=110., min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)
bus2 = pp.create_bus(net, vn_kv=110., min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)
bus3 = pp.create_bus(net, vn_kv=110., min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)

# create 110 kV lines
l1 = pp.create_line(net, bus1, bus2, length_km=1., std_type='149-AL1/24-ST1A 110.0')
l2 = pp.create_line(net, bus2, bus3, length_km=1., std_type='149-AL1/24-ST1A 110.0')
l3 = pp.create_line(net, bus3, bus1, length_km=1., std_type='149-AL1/24-ST1A 110.0')

# create loads
pp.create_load(net, bus3, p_mw=300)

# create generators
# p_mw is neglected while calculating optimal power flow, min_vm_pu matters
g1 = pp.create_gen(net, bus1, p_mw=200, min_vm_pu=300, controllable=True, slack=True)
g2 = pp.create_gen(net, bus2, p_mw=0, min_vm_pu=0, controllable=True)
g3 = pp.create_gen(net, bus3, p_mw=0, min_p_mw=0, max_p_mw=300, controllable=True)

In [31]:
# create cost functions for the generators
pp.create_poly_cost(net, element=g1, et="gen", cp1_eur_per_mw=30)
pp.create_poly_cost(net, element=g2, et="gen", cp1_eur_per_mw=30)
pp.create_poly_cost(net, element=g3, et="gen", cp1_eur_per_mw=29.999)

# run optimal power flow calculation
pp.runopp(net)
print(net.res_gen)

         p_mw    q_mvar     va_degree     vm_pu
0    0.029747 -0.032991  0.000000e+00  0.995924
1    0.029747 -0.032991  4.490908e-15  0.995924
2  299.940505 -0.032992 -5.840540e-05  0.995923


---